Clase Nivel Intermedio: Análisis Avanzado y Creación de Características en Datos de Tenis con Pandas
Objetivo: Que el estudiante sea capaz de manipular DataFrames de manera más avanzada, crear nuevas columnas relevantes para el análisis deportivo y generar visualizaciones que permitan obtener insights más profundos de los datos de tenis.

Duración: 1 hora y 30 minutos

Dataset: El mismo dataset atp_tennis.csv que utilizamos en la clase principiante.

Estructura de la Clase:

Repaso de la Clase Anterior y Preparación del Dataset (15 minutos):

Breve recapitulación de la carga, limpieza básica y consultas.
Carga del dataset y realización de la limpieza básica que aprendimos (eliminación de filas con valores nulos y conversión de la columna 'Date' a datetime).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Cargamos el dataset
df = pd.read_csv('atp_tennis.csv')

# Limpieza básica: eliminar filas con valores nulos
df_cleaned = df.dropna().copy() # Usamos .copy() para evitar warnings sobre modificaciones en la vista

# Convertimos la columna 'Date' a datetime
df_cleaned['Date'] = pd.to_datetime(df_cleaned['Date'])

print("Dataset limpio y con la columna 'Date' en formato datetime:")
print(df_cleaned.head())

.copy(): Crea una copia independiente del DataFrame. Esto es una buena práctica para evitar modificar el DataFrame original accidentalmente cuando se realizan operaciones.
Creación de Nuevas Características (Feature Engineering) (35 minutos):

Duración de los partidos (aproximación basada en el score):

Aunque no tenemos la duración exacta, podemos crear una característica aproximada basada en el número de sets jugados.

In [ ]:
# Función para contar el número de sets jugados basados en el 'Score'
def count_sets(score):
    if isinstance(score, str):
        return len(score.split())
    return None

# Aplicamos la función a la columna 'Score' para crear la columna 'Sets_Played'
df_cleaned['Sets_Played'] = df_cleaned['Score'].apply(count_sets)
print("\nPrimeras filas con la columna 'Sets_Played':")
print(df_cleaned[['Score', 'Sets_Played']].head())

# Visualizamos la distribución del número de sets jugados
plt.figure(figsize=(8, 6))
sns.countplot(x='Sets_Played', data=df_cleaned)
plt.title('Distribución del Número de Sets Jugados')
plt.xlabel('Número de Sets')
plt.ylabel('Número de Partidos')
plt.show()

def count_sets(score):: Define una función llamada count_sets que toma una cadena de texto score como argumento.
isinstance(score, str): Verifica si el argumento score es una cadena de texto. Esto es importante para manejar posibles valores no válidos en la columna.
score.split(): Si score es una cadena, este método divide la cadena en una lista de subcadenas utilizando el espacio como delimitador (ej: '6-3 7-5' se convierte en ['6-3', '7-5']). La longitud de esta lista nos da el número de sets.
df_cleaned['Score'].apply(count_sets): Aplica la función count_sets a cada valor de la columna 'Score' del DataFrame df_cleaned. El resultado (el número de sets para cada partido) se asigna a una nueva columna llamada 'Sets_Played'.
sns.countplot(x='Sets_Played', data=df_cleaned): Crea un gráfico de barras que muestra la frecuencia de cada valor en la columna 'Sets_Played'.
Diferencia de ranking entre jugadores:

In [ ]:
# Creamos una nueva columna con la diferencia de ranking
df_cleaned['Rank_Difference'] = abs(df_cleaned['Rank_1'] - df_cleaned['Rank_2'])
print("\nPrimeras filas con la columna 'Rank_Difference':")
print(df_cleaned[['Rank_1', 'Rank_2', 'Rank_Difference']].head())

# Visualizamos la distribución de la diferencia de ranking
plt.figure(figsize=(10, 6))
sns.histplot(df_cleaned['Rank_Difference'], bins=30, kde=True) # kde=True añade una estimación de la densidad del kernel
plt.title('Distribución de la Diferencia de Ranking entre Jugadores')
plt.xlabel('Diferencia de Ranking')
plt.ylabel('Frecuencia')
plt.show()

abs(): Función incorporada de Python que devuelve el valor absoluto de un número.
sns.histplot(...): Crea un histograma, que es una representación gráfica de la distribución de datos numéricos. bins especifica el número de barras (bins) en el histograma, y kde=True superpone una estimación de la densidad de probabilidad.
¿Jugó el mejor rankeado como local? (Asumiendo Player_1 como local - esto es una simplificación y podría no ser siempre cierto):

In [ ]:
# Creamos una columna booleana indicando si el jugador 1 tenía mejor ranking
df_cleaned['P1_Better_Ranked'] = df_cleaned['Rank_1'] < df_cleaned['Rank_2']
print("\nPrimeras filas con la columna 'P1_Better_Ranked':")
print(df_cleaned[['Rank_1', 'Rank_2', 'P1_Better_Ranked']].head())

# Contamos cuántas veces el jugador 1 tenía mejor ranking y ganó
better_ranked_p1_won = df_cleaned[(df_cleaned['P1_Better_Ranked'] == True) & (df_cleaned['Winner'] == df_cleaned['Player_1'])]
print(f"\nNúmero de partidos donde el jugador 1 tenía mejor ranking y ganó: {len(better_ranked_p1_won)}")

Al comparar directamente las columnas booleanas y de texto con ==, se crea una nueva Serie booleana que indica dónde la condición es verdadera.
Podemos combinar múltiples condiciones de filtrado usando los operadores lógicos & (and) y | (or).
Consultas y Agregaciones Más Complejas (25 minutos):

Encontrar el torneo con más partidos jugados:

In [ ]:
tournament_counts = df_cleaned['Tournament'].value_counts()
most_common_tournament = tournament_counts.idxmax() # Devuelve el índice con el valor máximo
count_most_common = tournament_counts.max()
print(f"\nEl torneo con más partidos jugados es '{most_common_tournament}' con {count_most_common} partidos.")

value_counts(): Como vimos antes, cuenta la frecuencia de cada valor en una columna.
.idxmax(): Devuelve el índice (en este caso, el nombre del torneo) que tiene el valor máximo en la Serie tournament_counts.
.max(): Devuelve el valor máximo en la Serie tournament_counts.
Ranking promedio de los ganadores por superficie:

In [ ]:
# Creamos una función para obtener el ranking del ganador
def get_winner_rank(row):
    if row['Winner'] == row['Player_1']:
        return row['Rank_1']
    elif row['Winner'] == row['Player_2']:
        return row['Rank_2']
    return None

df_cleaned['Winner_Rank'] = df_cleaned.apply(get_winner_rank, axis=1)

# Calculamos el ranking promedio de los ganadores por superficie
average_winner_rank_by_surface = df_cleaned.groupby('Surface')['Winner_Rank'].mean().sort_values()
print("\nRanking promedio de los ganadores por superficie:")
print(average_winner_rank_by_surface)

# Visualizamos esta información
plt.figure(figsize=(10, 6))
sns.barplot(x=average_winner_rank_by_surface.index, y=average_winner_rank_by_surface.values)
plt.title('Ranking Promedio de los Ganadores por Superficie')
plt.xlabel('Superficie')
plt.ylabel('Ranking Promedio del Ganador')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

df_cleaned.apply(get_winner_rank, axis=1): Aplica la función get_winner_rank a cada fila del DataFrame (axis=1). La función toma una fila como entrada y devuelve el ranking del ganador.
df_cleaned.groupby('Surface'): Agrupa el DataFrame por los valores únicos de la columna 'Surface'.
['Winner_Rank'].mean(): Calcula la media de la columna 'Winner_Rank' dentro de cada grupo (cada superficie).
.sort_values(): Ordena la Serie resultante por los valores del ranking promedio.
Visualizaciones Más Informativas (10 minutos):

Boxplot de la diferencia de ranking vs. el ganador:


In [ ]:
plt.figure(figsize=(10, 7))
sns.boxplot(x='Winner', y='Rank_Difference', data=df_cleaned)
plt.title('Diferencia de Ranking vs. Ganador del Partido')
plt.xlabel('Ganador')
plt.ylabel('Diferencia de Ranking')
plt.show()

sns.boxplot(...): Crea un diagrama de caja (boxplot), que muestra la distribución de una variable numérica para diferentes categorías. En este caso, vemos la distribución de la diferencia de ranking cuando el ganador fue el Jugador 1 versus cuando fue el Jugador 2.
Ejercicios para el Estudiante (10 minutos):

Crea una nueva columna que indique si el partido se jugó en cancha dura ('Hard').
Calcula el número promedio de sets jugados por superficie.
Encuentra los 5 torneos con la mayor diferencia promedio de ranking entre los jugadores.
Genera un gráfico de barras que muestre el número de partidos donde el 'Mejor de' fue 3 sets y donde fue 5 sets.
Cierre (5 minutos):

Resumen de la creación de nuevas características y consultas avanzadas.
Discusión sobre cómo estas técnicas pueden proporcionar insights más profundos en el análisis deportivo.
Introducción a la próxima clase (nivel avanzado).
¡Espero que esta clase sea muy enriquecedora para tu alumno! Avísame cuando estén listos para el nivel avanzado.

